In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/Yelp_Web_Scraper'

In [ ]:
biz_df = pd.read_csv(path+"/business_data.csv")
display(biz_df)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...
1,o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",NaN,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants
2,fNMVV_ZX7CJSDWQGdOM8Nw,"""Showmars Government Center""",Uptown,"""600 E 4th St""",Charlotte,NC,28202,35.221647,-80.839345,3.5,7,1,Restaurants;American (Traditional)
3,Gu-xs3NIQTj3Mj2xYoN2aw,"""Maxim Bakery & Restaurant""",NaN,"""9665 Bayview Avenue, Unit 1-4""",Richmond Hill,ON,L4C 9V4,43.867565,-79.412662,3.5,34,1,French;Food;Bakeries;Restaurants
4,1K4qrnfyzKzGgJPBEcJaNQ,"""Chula Taberna Mexicana""",Leslieville,"""1058 Gerrard Street E""",Toronto,ON,M4M 3A6,43.669256,-79.335902,3.5,39,1,Tiki Bars;Nightlife;Mexican;Restaurants;Bars
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40389,Ls_nR1MEcsOw5KuTlh0dfQ,"""Cole's Public House""",NaN,"""209 S Main St""",Amherst,OH,44001,41.398166,-82.226472,4.5,21,1,Bars;Gastropubs;Nightlife;Restaurants;Pubs;San...
40390,fukaxeFh8W9ijOp8sCrDyA,"""Heritage Fish & Chips""",NaN,"""1 Fisherman Drive""",Brampton,ON,L7A 2X9,43.714303,-79.799411,4.0,14,1,Seafood;Restaurants
40391,FxtrmM1a_0jcNnxiEF_Emg,"""Badgarten""",NaN,"""Mühlstr. 36""",Ludwigsburg,BW,71640,48.898590,9.201470,3.0,21,1,Restaurants;Beer Garden
40392,yJH_gq99aEj8xtyBaAQH8Q,"""Otto Onkel""",NaN,"""Böblinger Str. 14""",Stuttgart,BW,70178,48.764212,9.166086,3.0,52,1,German;Restaurants


In [ ]:
!curl ipecho.net/plain

34.73.204.88

In [ ]:
import requests
from lxml import html
import re
import time
import numpy as np
from random import shuffle, uniform
import json

def get_user_agents():
  return [
        'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
        'Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)',
        'curl/7.35.0',
        'Wget/1.15 (linux-gnu)',
        'Lynx/2.8.8pre.4 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/2.12.23'
  ]

def get_proxies():
    url = 'http://list.didsoft.com/get?email=scharfm16@gmail.com&pass=5sdegs&pid=http1000&showcountry=no'
    response = requests.get(url)
    proxies = response.text.split()
    return proxies


def get_url(city,rest):
    comb_raw = rest + ' ' + city
    comb_v2 = comb_raw.lower()
    comb = ' '.join(re.sub('[^0-9a-zA-Z ]+', '', comb_v2).split()).replace(' ','-')
    url = 'https://www.yelp.com/biz/' + comb
    return url

def get_pop_dish_list(row,proxies,user_agents):
    city = row['city']
    rest = row['name']

    url = get_url(city,rest)
    delay = uniform(.005,.015)
    time.sleep(delay)
    print(url)
    proxies_to_remove = []

    for proxy in proxies:
        shuffle(user_agents)
        user_agent = user_agents[0]

        username = r'scharfm16@gmail.com'
        password = r'5sdegs'

        proxy_http = 'http://' + username + ':' + password + '@' + proxy
        headers = {
            "User-Agent": user_agent,
            "ACCEPT": 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            "REFERER": 'https://www.google.com/',
            "ACCEPT-ENCODING": 'gzip, deflate, br'
            "PROXY": proxy_http,
        }

        try:
            r = requests.get(url,
                            headers=headers,
                             timeout=10)
            status = r.status_code
        except:
            status = 0
        print(status)
        
        if int(status) == 200:
            
            dom_tree = html.fromstring(r.content)

            # path='//p[@class={}]/text()'.format(class_name_raw)
            # dishes = dom_tree.xpath(path)
            
            a = str([l for l in dom_tree.xpath('//text()') if '!--{"gaConfig"' in l][0][4:-3])
            json_tree = json.loads(a)
            try:
              popular_dish_dict = json_tree['bizDetailsPageProps']['popularDishesCarouselProps']['popularDishes']
              if popular_dish_dict == None:
                dishes = []
              else:
                dishes = [dish['dishName'] for dish in popular_dish_dict]
            except TypeError:
              dishes = []
            print(dishes)
            if len(dishes) == 0:
                return np.nan, proxies_to_remove
            else:
                return dishes, proxies_to_remove
        elif int(status) == 404:
            return np.nan, proxies_to_remove
        else:
            print("Proxy failure:",proxy)
            print("user agent:",user_agent)
            proxies_to_remove.append(proxy)
    return 'break', proxies_to_remove

In [ ]:
old_dish = pd.read_csv(path+'/dish_data.csv')
length = len(biz_df)
start_idx = len(old_dish)
dish_list = []
display(old_dish)

,restaurant,dishes
0,PfOCPjBrlQAnz__NXj9h_w,"['Drunken Pork Chops', 'Brick House Famous BBQ..."
1,o9eMRCWt5PkpLDE0gOPtcQ,NaN
2,fNMVV_ZX7CJSDWQGdOM8Nw,NaN
3,Gu-xs3NIQTj3Mj2xYoN2aw,NaN
4,1K4qrnfyzKzGgJPBEcJaNQ,"['Shrimp Tacos', 'Churros', 'Fish Tacos', 'Bee..."
...,...,...
40389,Ls_nR1MEcsOw5KuTlh0dfQ,"['Baked Goat', 'Meatloaf', 'Fried Corn Fritter..."
40390,fukaxeFh8W9ijOp8sCrDyA,NaN
40391,FxtrmM1a_0jcNnxiEF_Emg,NaN
40392,yJH_gq99aEj8xtyBaAQH8Q,NaN


In [ ]:
print("Getting proxies")
proxies = get_proxies()
proxies = ['ye','yee','yee']
user_agents = get_user_agents()

print("Starting iteration")
print('Number of proxies:',len(proxies))
for idx, row in biz_df.iloc[start_idx:].iterrows():
    shuffle(proxies)
    dishes, proxies_to_remove = get_pop_dish_list(row,proxies,user_agents)
    if dishes == 'break':
        print("All proxies failed: aborting")
        break
    else:  
        output = (row['business_id'],dishes)
        proxies = list(set(proxies) - set(proxies_to_remove))
        dish_list.append(output)

        print('Number of proxies:',len(proxies))
        print(idx/length,'of the way done') 

Getting proxies
Starting iteration
Number of proxies: 3


In [ ]:
new_dish = pd.DataFrame(dish_list,columns=['restaurant','dishes'])
dish_df = old_dish.append(new_dish)
display(dish_df)

,restaurant,dishes
0,PfOCPjBrlQAnz__NXj9h_w,"['Drunken Pork Chops', 'Brick House Famous BBQ..."
1,o9eMRCWt5PkpLDE0gOPtcQ,NaN
2,fNMVV_ZX7CJSDWQGdOM8Nw,NaN
3,Gu-xs3NIQTj3Mj2xYoN2aw,NaN
4,1K4qrnfyzKzGgJPBEcJaNQ,"['Shrimp Tacos', 'Churros', 'Fish Tacos', 'Bee..."
...,...,...
40389,Ls_nR1MEcsOw5KuTlh0dfQ,"['Baked Goat', 'Meatloaf', 'Fried Corn Fritter..."
40390,fukaxeFh8W9ijOp8sCrDyA,NaN
40391,FxtrmM1a_0jcNnxiEF_Emg,NaN
40392,yJH_gq99aEj8xtyBaAQH8Q,NaN


In [ ]:
dish_df.to_csv(path+'/dish_data.csv',index=False)